In [1]:
#Used Libraries
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
#1. How many users got paid?

queryResult("""
SELECT
    COUNT(DISTINCT(user_id))
FROM
    payout
""")

,count
0,433


In [3]:
#2. Show the 5 users with the highest payouts

queryResult("""
SELECT
    user_id,
    SUM(payout)
FROM
    payout
GROUP BY
    user_id
ORDER BY
    SUM(payout) DESC
LIMIT 5;
""")


,user_id,sum
0,1537,10.74169
1,3051,9.73121
2,1512,8.80816
3,1127,8.78861
4,4848,8.42445


In [4]:
#3. Show the 5 users with the lowest payouts

queryResult("""
SELECT
    user_id,
    SUM(payout)
FROM
    payout
GROUP BY
    user_id
ORDER BY
    SUM(payout)
LIMIT 5;
""")


,user_id,sum
0,2003,0.00775
1,3410,0.00818
2,2813,0.02914
3,2462,0.03381
4,4467,0.05056


In [5]:
#4. How much ether was paid out in November 2020?

queryResult("""
SELECT
    EXTRACT(MONTH FROM date) AS month,
    SUM(payout)
FROM
    payout
WHERE
    EXTRACT(MONTH FROM date) = 11
GROUP BY
    month
""")

,month,sum
0,11.0,166.0118


In [6]:
#5. Which plan is the most popular?

queryResult("""
SELECT
    "Plan",
    COUNT(DISTINCT(user_id))
FROM
    plan
GROUP BY
    "Plan"
ORDER BY
    COUNT(DISTINCT(user_id)) DESC
LIMIT 1;
""")

,Plan,count
0,Free,220


The Free plan

In [7]:
#6. Which plan is the most popular amongst Linux users?

queryResult("""
SELECT
    "Plan",
    "OS",
    COUNT(DISTINCT(user_id))
FROM
    plan
GROUP BY
    "Plan",
    "OS"
ORDER BY 
    "OS",
    COUNT(DISTINCT(user_id)) DESC
LIMIT 1
""")

,Plan,OS,count
0,Premium,Linux,76


The Premium plan

In [8]:
#7. What is the percentage of payout between the different plans?

queryResult("""
WITH 
    total as (SELECT sum(payout) as total
FROM 
    payout)
SELECT 
    plan."Plan",
    sum(payout.payout / total.total) as percentage
FROM
    plan
INNER JOIN payout ON plan.user_id = payout.user_id,
    total
GROUP BY
    plan."Plan"
""")


,Plan,percentage
0,Free,0.53972
1,Premium,0.46028


In [9]:
#8. Users of which operating system earned more in payouts?

queryResult("""
SELECT
    COUNT(DISTINCT(payout.user_id)) AS users,
    plan."OS",
    SUM(payout.payout) AS sum_payout,
    COUNT(DISTINCT(payout.user_id))/SUM(payout.payout) AS avg_payout
FROM
    payout
LEFT JOIN plan ON payout.user_id = plan.user_id
GROUP BY
    plan."OS"
ORDER BY
    avg_payout DESC;
    
""")

,users,OS,sum_payout,avg_payout
0,141,MAC,267.41123,0.527278
1,148,Windows,316.03701,0.468300
2,144,Linux,322.00353,0.447200


In [10]:
#9. What is the average payout amount per user for each of the OS in July 2020?

queryResult("""
SELECT
    COUNT(DISTINCT(payout.user_id)) AS users,
    EXTRACT(MONTH FROM payout.date) AS month,
    plan."OS",
    SUM(payout.payout) AS sum_payout,
    AVG(payout.payout) AS avg_payout
FROM
    payout
LEFT JOIN plan ON payout.user_id = plan.user_id
WHERE
    EXTRACT(MONTH FROM date) = 7
GROUP BY
    plan."OS",
    EXTRACT(MONTH FROM payout.date)
ORDER BY
    avg_payout DESC;
    
""")

,users,month,OS,sum_payout,avg_payout
0,30,7.0,Linux,61.79566,1.931114
1,20,7.0,MAC,36.09181,1.640537
2,22,7.0,Windows,38.17441,1.590600


In [14]:
#10. What is the daily share of ether earned by users from Linux that are in the free plan in this data?

queryResult("""
WITH 
    total as (
    SELECT 
        date,
        sum(payout) as daily_total
    FROM 
        payout
    GROUP BY
        date)
SELECT
    COUNT(plan.user_id),
    plan."OS",
    plan."Plan",
    payout.date,
    SUM(payout.payout) AS sum_payout,
    SUM(total.daily_total) AS daily_total,
    SUM(payout.payout)/SUM(total.daily_total) as daily_share
FROM
    payout
        INNER JOIN plan ON payout.user_id = plan.user_id
        INNER JOIN total ON payout.date = total.date
WHERE
    plan."OS" = 'Linux' AND plan."Plan" = 'Free' 
GROUP BY
    plan."OS",
    plan."Plan",
    payout.date
ORDER BY
    daily_share DESC
""")

,count,OS,Plan,date,sum_payout,daily_total,daily_share
0,1,Linux,Free,2020-12-07,0.97608,0.97608,1.000000
1,1,Linux,Free,2020-07-28,2.78539,2.78539,1.000000
2,1,Linux,Free,2020-10-23,0.29519,0.29519,1.000000
3,1,Linux,Free,2020-09-29,0.81951,0.81951,1.000000
4,1,Linux,Free,2020-12-22,1.11205,1.11205,1.000000
...,...,...,...,...,...,...,...
73,1,Linux,Free,2020-08-02,0.49698,8.76264,0.056716
74,2,Linux,Free,2020-11-23,0.80015,15.10014,0.052990
75,1,Linux,Free,2020-11-24,0.22961,7.86548,0.029192
76,1,Linux,Free,2020-10-14,0.13800,6.89914,0.020002
